# Máquinas de Boltzmann Restringidas

In [1]:
import numpy as np
import pandas as pd
import torch

In [2]:
from google.colab import drive
drive.mount("/content/drive")

%cd "/content/drive/MyDrive/deeplearning-az/Notebooks/Part 5 - Boltzmann Machines (BM)"

Mounted at /content/drive
/content/drive/MyDrive/deeplearning-az/Notebooks/Part 5 - Boltzmann Machines (BM)


## Importar el dataset

In [3]:
movies = pd.read_csv("ml-1m/movies.dat", sep="::", header=None, engine="python", encoding="latin-1")
users = pd.read_csv("ml-1m/users.dat", sep="::", header=None, engine="python", encoding="latin-1")
ratings = pd.read_csv("ml-1m/ratings.dat", sep="::", header=None, engine="python", encoding="latin-1")

## Preparar el conjunto de entrenamiento y el conjunto de testing

In [4]:
training_set = pd.read_csv("ml-100k/u1.base", sep="\t", header=None)
training_set = np.array(training_set, dtype="int")
test_set = pd.read_csv("ml-100k/u1.test", sep="\t", header=None)
test_set = np.array(test_set, dtype="int")

## Obtener el número de usuarios y de películas

In [5]:
nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))

## Convertir los datos en un array $X_{[u,i]}$ con usuarios en $(u)$ filas y películas en $(i)$ columnas

In [6]:
def convert(data):
    new_data = []
    for id_user in range(1, nb_users + 1):
        id_movies = data[:, 1][data[:, 0] == id_user]
        id_ratings = data[:, 2][data[:, 0] == id_user]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [7]:
training_set = convert(training_set)
test_set = convert(test_set)


## Convertir los datos a tensores de Torch

In [8]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Convertir las valoraciones a valores binarios 1 (Me gusta) o 0 (No me gusta)

In [9]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

In [10]:
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

## Crear la arquitectura de la Red Neuronal (Modelo Probabilistico Gráfico)

In [11]:
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)
    def sample_h(self, x):  # x = mini_batch_size x nv
        wx = torch.mm(x, self.W.t())  # mini_batch_size x nh
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    def sample_v(self, y):  # y = mini_batch_size x nh
        wy = torch.mm(y, self.W)  # mini_batch_size x nv
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    def train(self, v0, vk, ph0, phk):
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)

In [12]:
nv = len(training_set[0])
nh = 100

rbm = RBM(nv, nh)

## Entrenar la RBM en base a la `distancia promedio`

In [13]:
nb_epoch = 10
batch_size = 100
for epoch in range(1, nb_epoch + 1):
    training_loss = 0
    s = 0.0
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user + batch_size]
        v0 = training_set[id_user:id_user + batch_size]
        ph0, _ = rbm.sample_h(v0)
        for k in range(10):
            _, hk = rbm.sample_h(vk)
            _, vk = rbm.sample_v(hk)
            vk[v0 < 0] = v0[v0 < 0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        training_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))  # Distancia promedio
        s += 1.0
    print("Epoch: " + str(epoch) + ", Loss: " + str(training_loss/s))

Epoch: 1, Loss: tensor(0.3439)
Epoch: 2, Loss: tensor(0.2504)
Epoch: 3, Loss: tensor(0.2514)
Epoch: 4, Loss: tensor(0.2516)
Epoch: 5, Loss: tensor(0.2480)
Epoch: 6, Loss: tensor(0.2475)
Epoch: 7, Loss: tensor(0.2477)
Epoch: 8, Loss: tensor(0.2466)
Epoch: 9, Loss: tensor(0.2503)
Epoch: 10, Loss: tensor(0.2486)


### Testear la RBM en base a la `distancia promedio`

In [14]:
testing_loss = 0
s = 0.0
for id_user in range(nb_users):
    v = training_set[id_user:id_user + 1]
    vt = test_set[id_user:id_user + 1]
    if len(vt[vt >= 0]) > 0:
        _, h = rbm.sample_h(v)
        _, v = rbm.sample_v(h)
        testing_loss += torch.mean(torch.abs(vt[vt >= 0] - v[vt >= 0]))  # Distancia promedio
        s += 1.0
        print("Testing Loss: " + str(testing_loss/s))

Testing Loss: tensor(0.2117)
Testing Loss: tensor(0.2877)
Testing Loss: tensor(0.3584)
Testing Loss: tensor(0.3438)
Testing Loss: tensor(0.3346)
Testing Loss: tensor(0.3019)
Testing Loss: tensor(0.2889)
Testing Loss: tensor(0.2700)
Testing Loss: tensor(0.2622)
Testing Loss: tensor(0.2471)
Testing Loss: tensor(0.2498)
Testing Loss: tensor(0.2354)
Testing Loss: tensor(0.2412)
Testing Loss: tensor(0.2353)
Testing Loss: tensor(0.2514)
Testing Loss: tensor(0.2480)
Testing Loss: tensor(0.2530)
Testing Loss: tensor(0.2428)
Testing Loss: tensor(0.2405)
Testing Loss: tensor(0.2489)
Testing Loss: tensor(0.2569)
Testing Loss: tensor(0.2562)
Testing Loss: tensor(0.2554)
Testing Loss: tensor(0.2479)
Testing Loss: tensor(0.2412)
Testing Loss: tensor(0.2451)
Testing Loss: tensor(0.2459)
Testing Loss: tensor(0.2415)
Testing Loss: tensor(0.2454)
Testing Loss: tensor(0.2502)
Testing Loss: tensor(0.2520)
Testing Loss: tensor(0.2491)
Testing Loss: tensor(0.2476)
Testing Loss: tensor(0.2462)
Testing Loss: 

In [15]:
# Comprobar como 0.25 de distancia promedio se corresponde con una tasa del 75% de éxito

u = np.random.choice([0, 1], 100000)
v = np.random.choice([0, 1], 100000)
u[:50000] = v[:50000]
sum(u==v)/float(len(u)) # -> obtendremos 0.75
np.mean(np.abs(u-v)) # -> obtendremos 0.25

0.24981

## Entrenar la RBM en base al `RMSE`

In [16]:
nb_epoch = 10
batch_size = 100
for epoch in range(1, nb_epoch + 1):
    training_loss = 0
    s = 0.0
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user + batch_size]
        v0 = training_set[id_user:id_user + batch_size]
        ph0, _ = rbm.sample_h(v0)
        for k in range(10):
            _, hk = rbm.sample_h(vk)
            _, vk = rbm.sample_v(hk)
            vk[v0 < 0] = v0[v0 < 0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        training_loss += np.sqrt(torch.mean((v0[v0 >= 0] - vk[v0 >= 0])**2))  # RMSE
        s += 1.0
    print("Epoch: " + str(epoch) + ", Loss: " + str(training_loss/s))

Epoch: 1, Loss: tensor(0.4993)
Epoch: 2, Loss: tensor(0.4958)
Epoch: 3, Loss: tensor(0.4962)
Epoch: 4, Loss: tensor(0.4953)
Epoch: 5, Loss: tensor(0.4973)
Epoch: 6, Loss: tensor(0.4986)
Epoch: 7, Loss: tensor(0.4989)
Epoch: 8, Loss: tensor(0.4977)
Epoch: 9, Loss: tensor(0.4973)
Epoch: 10, Loss: tensor(0.4997)


### Testear la RBM en base al `RMSE`

In [17]:
testing_loss = 0
s = 0.0
for id_user in range(nb_users):
    v = training_set[id_user:id_user + 1]
    vt = test_set[id_user:id_user + 1]
    if len(vt[vt >= 0]) > 0:
        _, h = rbm.sample_h(v)
        _, v = rbm.sample_v(h)
        testing_loss += np.sqrt(torch.mean((vt[vt >= 0] - v[vt >= 0])**2))  # RMSE
        s += 1.0
        print("Testing Loss: " + str(testing_loss/s))

Testing Loss: tensor(0.4680)
Testing Loss: tensor(0.4723)
Testing Loss: tensor(0.5595)
Testing Loss: tensor(0.5314)
Testing Loss: tensor(0.5561)
Testing Loss: tensor(0.5338)
Testing Loss: tensor(0.5214)
Testing Loss: tensor(0.5131)
Testing Loss: tensor(0.5058)
Testing Loss: tensor(0.4901)
Testing Loss: tensor(0.4977)
Testing Loss: tensor(0.4794)
Testing Loss: tensor(0.4849)
Testing Loss: tensor(0.4802)
Testing Loss: tensor(0.4942)
Testing Loss: tensor(0.4901)
Testing Loss: tensor(0.5005)
Testing Loss: tensor(0.4938)
Testing Loss: tensor(0.4913)
Testing Loss: tensor(0.5005)
Testing Loss: tensor(0.5051)
Testing Loss: tensor(0.5060)
Testing Loss: tensor(0.5013)
Testing Loss: tensor(0.4943)
Testing Loss: tensor(0.4892)
Testing Loss: tensor(0.4912)
Testing Loss: tensor(0.4922)
Testing Loss: tensor(0.4925)
Testing Loss: tensor(0.4960)
Testing Loss: tensor(0.4987)
Testing Loss: tensor(0.5045)
Testing Loss: tensor(0.5031)
Testing Loss: tensor(0.5014)
Testing Loss: tensor(0.4959)
Testing Loss: 